# 1. Write a python program which searches all the product under a particular product from www.amazon.in.
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search
for guitars.

In [16]:
#import all the required libraries
import pandas as pd
import selenium
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import time
import re

In [9]:
driver.get('https://www.amazon.in/')

In [10]:
inputU = input('please enter product here--->')

please enter product here--->guitars


In [11]:
search_bar = driver.find_element(By.XPATH,'//*[@id="twotabsearchtextbox"]')    # Finding the search bar using it's xpath
search_bar.send_keys(inputU)       # Inputing keyword to search 
search_button = driver.find_element(By.XPATH,'//*[@id="nav-search-submit-button"]')    # Finding the xpath of search button
search_button.click()        # Clicking the search butt

In [12]:
productName=[]

In [14]:
#scraping the Product_Name 
PName=driver.find_elements(By.XPATH,"//span[@class='a-size-medium a-color-base a-text-normal']")
for i in PName:
    if i.text is None :
        productName.append("--") 
    else:
        productName.append(i.text)
print(len(productName),productName)

2 ['Kadence Guitar Acoustica Series, 41” Jumbo Size 6-strings Acoustic Guitar, Spruce body with Pick Guard A311 with bag… B07JKYWGJN (KAD-A-311)', 'Kadence Guitar Acoustica Series, Electric Acoustic Guitar, Ash Wood with Pickup and Inbuilt tuner (Ash Wood, Electro Acoustic) with bag.']


# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search
results then scrape all the products available under that product name. Details to be scraped are: "Brand
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.


In [27]:
start_page = 0
end_page = 3
urls = []
for page in range(start_page,end_page+1):
    try:
        page_urls = driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-no-outline"]')
        
        # appending all the urls on current page to urls list
        for url in page_urls:
            url = url.get_attribute('href')     # Scraping the url from webelement
            if url[0:4]=='http':                # Checking if the scraped data is a valid url or not
                urls.append(url)                # Appending the url to urls list
        print("Product urls of page {} has been scraped.".format(page+1))
        
        # Moving to next page
        nxt_button = driver.find_elements(By.XPATH,'//li[@class="a-last"]/a')      # Locating the next_button which is active
        if nxt_button.text == 'Next→':                                            # Checking if the button located is next button
            nxt_button.click()                                                    # Clicking the next button
            time.sleep(5)                                                         # time delay of 5 seconds
        # If the current active button is not next button, the we will check if the next button is inactive or not    
        elif driver.find_element(By.XPATH,'//li[@class="a-disabled a-last"]/a').text == 'Next→':    
            print("No new pages exist. Breaking the loop")  # Printing message and breakinf loop if we have reached the last page
            break
            
    except StaleElementReferenceException as e:             # Handling StaleElement Exception   
        print("Stale Exception")
        next_page = nxt_button.get_attribute('href')        # Extracting the url of next page
        driver.get(next_page)                               # ReLoading the next page

Product urls of page 1 has been scraped.


AttributeError: 'list' object has no attribute 'text'

In [28]:
prod_dict = {}
prod_dict['Brand']=[]
prod_dict['Name']=[]
prod_dict['Rating']=[]
prod_dict['No. of ratings']=[]
prod_dict['Price']=[]
prod_dict['Return/Exchange']=[]
prod_dict['Expected Delivery']=[] 
prod_dict['Availability']=[]
prod_dict['Other Details']=[]
prod_dict['URL']=[]

In [29]:
for url in urls[:4]:
    driver.get(url)                                                        # Loading the webpage by url
    print("Scraping URL = ", url)
    #time.sleep(2)
    
    try:
        brand = driver.find_element_by_xpath('//a[@id="bylineInfo"]')      # Extracting Brand from xpath
        prod_dict['Brand'].append(brand.text)
    except NoSuchElementException:
        prod_dict['Brand'].append('-')
    
    try:
        name = driver.find_element_by_xpath('//h1[@id="title"]/span')      # Extracting Name from xpath
        prod_dict['Name'].append(name.text)
    except NoSuchElementException:
        prod_dict['Name'].append('-')
    
    try:
        rating = driver.find_element_by_xpath('//span[@id="acrPopover"]')  # Extracting Ratings from xpath
        prod_dict['Rating'].append(rating.get_attribute("title"))
    except NoSuchElementException:
        prod_dict['Rating'].append('-')
    
    try:
        n_rating = driver.find_element_by_xpath('//a[@id="acrCustomerReviewLink"]/span')     # Extracting no. of Ratings from xpath
        prod_dict['No. of ratings'].append(n_rating.text)
    except NoSuchElementException:
        prod_dict['No. of ratings'].append('-')
    
    try:
        price = driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]')            # Extracting Price from xpath
        prod_dict['Price'].append(price.text)
    except NoSuchElementException:
        prod_dict['Price'].append('-')
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        ret = driver.find_element_by_xpath('//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        prod_dict['Return/Exchange'].append(ret.text)
    except NoSuchElementException:
        prod_dict['Return/Exchange'].append('-')
    try:
        delivry = driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b')         # Extracting Expected Delivery from xpath
        prod_dict['Expected Delivery'].append(delivry.text)
    except NoSuchElementException:
        prod_dict['Expected Delivery'].append('-')
    
    try:
        avl = driver.find_element_by_xpath('//div[@id="availability"]/span')                # Extracting Availability from xpath
        prod_dict['Availability'].append(avl.text)
    except NoSuchElementException:
        prod_dict['Availability'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        dtls = driver.find_element_by_xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]')
        prod_dict['Other Details'].append('  ||  '.join(dtls.text.split('\n')))
    except NoSuchElementException:
         prod_dict['Other Details'].append('-')
    
    prod_dict['URL'].append(url)                                                            # Saving url
    time.sleep(2)

In [30]:
prod_df = pd.DataFrame.from_dict(prod_dict)
prod_df

,Brand,Name,Rating,No. of ratings,Price,Return/Exchange,Expected Delivery,Availability,Other Details,URL


In [31]:
#saving data to csv
prod_df.to_csv('Amazon_{}.csv'.format(inputU))

# 3. Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [1]:
import selenium
from selenium import webdriver # ready to start the browser
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from selenium.webdriver.common.by  import By

import time


In [2]:
# first connect to the driver

driver = webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver_win32\chromedriver.exe")

# connect to  the url that i want to open 
driver.get("https://images.google.com/")

In [3]:
Product=driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/div')# absolute x_path
Product.send_keys("fruits")

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=107.0.5304.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x0056ACD3+2075859]
	Ordinal0 [0x004FEE61+1633889]
	Ordinal0 [0x003FB680+571008]
	Ordinal0 [0x0042647E+746622]
	Ordinal0 [0x00425B5A+744282]
	Ordinal0 [0x0044731C+881436]
	Ordinal0 [0x004215BF+726463]
	Ordinal0 [0x00447534+881972]
	Ordinal0 [0x0045B56A+963946]
	Ordinal0 [0x00447136+880950]
	Ordinal0 [0x0041FEFD+720637]
	Ordinal0 [0x00420F3F+724799]
	GetHandleVerifier [0x0081EED2+2769538]
	GetHandleVerifier [0x00810D95+2711877]
	GetHandleVerifier [0x005FA03A+521194]
	GetHandleVerifier [0x005F8DA0+516432]
	Ordinal0 [0x0050682C+1665068]
	Ordinal0 [0x0050B128+1683752]
	Ordinal0 [0x0050B215+1683989]
	Ordinal0 [0x00516484+1729668]
	BaseThreadInitThunk [0x76A0FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77297A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77297A6E+238]


In [ ]:
search_button=driver.find_element(By.XPATH,'//*[@id="sbtc"]/button')# absolute x_path
search_button.click()

In [ ]:
print("start scrolling to generate more images on the page...")
# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")

In [ ]:
images = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

In [ ]:
img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)

In [ ]:
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open("H:/Flip ROBO/banana/img"+str(i)+".jpg", "wb")
    file.write(response.content)

4  .Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [ ]:
https://github.com/Anwesh91/FlipRobo/blob/main/FlipRobo_Webscrapping_3_All_Updated.ipynb

In [1]:
import selenium
from selenium import webdriver # ready to start the browser
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from selenium.webdriver.common.by  import By

import time

In [4]:
# first connect to the driver

driver = webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver_win32\chromedriver.exe")

# connect to  the url that i want to open 
driver.get("https://www.flipkart.com/")

In [3]:
Product=driver.find_element(By.XPATH,"/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input ")# absolute x_path
Product.send_keys("smartphone") 

In [4]:
search_button=driver.find_element(By.XPATH,'//button[@class="L0Z3Pu"]')# absolute x_path
search_button.click()

In [5]:
# create empty list whatevwe we want to scrap
Brand_Name=[]
mobile_Colour=[]
Storage_RAM_ROM=[]
Primary_Secondary_Camera=[]
Display_Size=[]
Battery_Capacity=[]
mobile_Price=[]
Product_URL=[]

In [6]:
#scraping the BrandName
BrandName=driver.find_elements(By.XPATH,'//div[@class="_4rR01T"]')# relative x_path
for i in BrandName:
    if i.text is None :
        Brand_Name.append("-") 
    else:
        Brand_Name.append(i.text)
print(len(Brand_Name),Brand_Name)

24 ['SAMSUNG Galaxy F13 (Waterfall Blue, 64 GB)', 'SAMSUNG Galaxy F13 (Nightsky Green, 64 GB)', 'APPLE iPhone 11 (White, 128 GB)', 'APPLE iPhone 11 (Black, 128 GB)', 'SAMSUNG Galaxy F13 (Sunrise Copper, 64 GB)', 'realme 9i 5G (Metallica Gold, 64 GB)', 'POCO C31 (Royal Blue, 64 GB)', 'realme C33 (Sandy Gold, 64 GB)', 'REDMI 10 (Pacific Blue, 64 GB)', 'realme C33 (Aqua Blue, 32 GB)', 'realme C33 (Night Sea, 64 GB)', 'realme C33 (Aqua Blue, 64 GB)', 'realme C33 (Sandy Gold, 32 GB)', 'realme C33 (Night Sea, 32 GB)', 'REDMI 10 (Midnight Black, 64 GB)', 'APPLE iPhone 11 (Black, 64 GB)', 'realme C30 (Denim Black, 32 GB)', 'realme C30 (Bamboo Green, 32 GB)', 'realme C30 (Lake Blue, 32 GB)', 'REDMI 10 (Caribbean Green, 64 GB)', 'REDMI 9i Sport (Coral Green, 64 GB)', 'APPLE iPhone 13 (Pink, 128 GB)', 'REDMI 9i Sport (Metallic Blue, 64 GB)', 'APPLE iPhone 11 (Red, 64 GB)']


In [7]:
#scraping the Storage
ram_rom=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[1]")# relative x_path
for i in ram_rom:
    if i.text is None :
        Storage_RAM_ROM.append("-") 
    else:
        Storage_RAM_ROM.append(i.text)
print(len(Storage_RAM_ROM),Storage_RAM_ROM)

24 ['4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '128 GB ROM', '128 GB ROM', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 512 GB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '3 GB RAM | 32 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '3 GB RAM | 32 GB ROM | Expandable Upto 1 TB', '3 GB RAM | 32 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '64 GB ROM', '2 GB RAM | 32 GB ROM | Expandable Upto 1 TB', '2 GB RAM | 32 GB ROM | Expandable Upto 1 TB', '2 GB RAM | 32 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 512 GB', '128 GB ROM', '4 GB RAM | 64 GB ROM | Expandable Upto 512 GB', '64 GB ROM']


In [8]:
#scraping the camera
camera=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[3]")# relative x_path
for i in camera:
    if i.text is None :
        Primary_Secondary_Camera.append("-") 
    else:
        Primary_Secondary_Camera.append(i.text)
print(len(Primary_Secondary_Camera),Primary_Secondary_Camera)

24 ['50MP + 5MP + 2MP | 8MP Front Camera', '50MP + 5MP + 2MP | 8MP Front Camera', '12MP + 12MP | 12MP Front Camera', '12MP + 12MP | 12MP Front Camera', '50MP + 5MP + 2MP | 8MP Front Camera', '50MP + 2MP + 2MP | 8MP Front Camera', '13MP + 2MP + 2MP | 5MP Front Camera', '50MP + 0.3MP | 5MP Front Camera', '50MP + 2MP | 5MP Front Camera', '50MP + 0.3MP | 5MP Front Camera', '50MP + 0.3MP | 5MP Front Camera', '50MP + 0.3MP | 5MP Front Camera', '50MP + 0.3MP | 5MP Front Camera', '50MP + 0.3MP | 5MP Front Camera', '50MP + 2MP | 5MP Front Camera', '12MP + 12MP | 12MP Front Camera', '8MP Rear Camera | 5MP Front Camera', '8MP Rear Camera | 5MP Front Camera', '8MP Rear Camera | 5MP Front Camera', '50MP + 2MP | 5MP Front Camera', '13MP Rear Camera | 5MP Front Camera', '12MP + 12MP | 12MP Front Camera', '13MP Rear Camera | 5MP Front Camera', '12MP + 12MP | 12MP Front Camera']


In [9]:
#scraping the  Battery_Capacity
B=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[4]")# relative x_path
for i in B:
    if i.text is None :
        Battery_Capacity.append("-") 
    else:
        Battery_Capacity.append(i.text)
print(len(Battery_Capacity),Battery_Capacity)

24 ['6000 mAh Lithium Ion Battery', '6000 mAh Lithium Ion Battery', 'A13 Bionic Chip Processor', 'A13 Bionic Chip Processor', '6000 mAh Lithium Ion Battery', '5000 mAh Lithium Ion Battery', '5000 mAh Lithium-ion Polymer Battery', '5000 mAh Lithium Ion Battery', '6000 mAh Lithium Polymer Battery', '5000 mAh Lithium Ion Battery', '5000 mAh Lithium Ion Battery', '5000 mAh Lithium Ion Battery', '5000 mAh Lithium Ion Battery', '5000 mAh Lithium Ion Battery', '6000 mAh Lithium Polymer Battery', 'A13 Bionic Chip Processor', '5000 mAh Lithium Ion Battery', '5000 mAh Lithium Ion Battery', '5000 mAh Lithium Ion Battery', '6000 mAh Lithium Polymer Battery', '5000 mAh Li-Polymer Battery', 'A15 Bionic Chip Processor', '5000 mAh Li-Polymer Battery', 'A13 Bionic Chip Processor']


In [10]:
#scraping the Display_Size
ds=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[2]")# relative x_path
for i in ds:
    if i.text is None :
        Display_Size.append("-") 
    else:
        Display_Size.append(i.text)
print(len(Display_Size),Display_Size)

24 ['16.76 cm (6.6 inch) Full HD+ Display', '16.76 cm (6.6 inch) Full HD+ Display', '15.49 cm (6.1 inch) Liquid Retina HD Display', '15.49 cm (6.1 inch) Liquid Retina HD Display', '16.76 cm (6.6 inch) Full HD+ Display', '16.76 cm (6.6 inch) Full HD+ Display', '16.59 cm (6.53 inch) HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '17.02 cm (6.7 inch) HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '17.02 cm (6.7 inch) HD+ Display', '15.49 cm (6.1 inch) Liquid Retina HD Display', '16.51 cm (6.5 inch) HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '16.51 cm (6.5 inch) HD+ Display', '17.02 cm (6.7 inch) HD+ Display', '16.59 cm (6.53 inch) HD+ Display', '15.49 cm (6.1 inch) Super Retina XDR Display', '16.59 cm (6.53 inch) HD+ Display', '15.49 cm (6.1 inch) Liquid Retina HD Display']


In [11]:
#scraping the mobile_Price
price=driver.find_elements(By.XPATH,"//div[@class='_30jeq3 _1_WHN1']")# relative x_path
for i in price:
    if i.text is None :
        mobile_Price.append("-") 
    else:
        mobile_Price.append(i.text)
print(len(mobile_Price),mobile_Price)

24 ['₹9,499', '₹9,499', '₹41,990', '₹41,990', '₹9,499', '₹12,999', '₹7,499', '₹9,999', '₹8,999', '₹8,999', '₹9,999', '₹9,999', '₹8,999', '₹8,999', '₹8,999', '₹35,990', '₹5,699', '₹5,699', '₹5,699', '₹8,999', '₹7,099', '₹59,990', '₹7,099', '₹34,990']


In [12]:
FlipKart=pd.DataFrame([])
FlipKart['Brand_Name']=Brand_Name
FlipKart['Storage_RAM_ROM']=Storage_RAM_ROM
FlipKart['Primary_Secondary_Camera']=Primary_Secondary_Camera
FlipKart['Display_Size']=Display_Size
FlipKart['Battery_Capacity']=Battery_Capacity
FlipKart['mobile_Price']=mobile_Price


In [13]:
FlipKart

,Brand_Name,Storage_RAM_ROM,Primary_Secondary_Camera,Display_Size,Battery_Capacity,mobile_Price
0,"SAMSUNG Galaxy F13 (Waterfall Blue, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,50MP + 5MP + 2MP | 8MP Front Camera,16.76 cm (6.6 inch) Full HD+ Display,6000 mAh Lithium Ion Battery,"₹9,499"
1,"SAMSUNG Galaxy F13 (Nightsky Green, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,50MP + 5MP + 2MP | 8MP Front Camera,16.76 cm (6.6 inch) Full HD+ Display,6000 mAh Lithium Ion Battery,"₹9,499"
2,"APPLE iPhone 11 (White, 128 GB)",128 GB ROM,12MP + 12MP | 12MP Front Camera,15.49 cm (6.1 inch) Liquid Retina HD Display,A13 Bionic Chip Processor,"₹41,990"
3,"APPLE iPhone 11 (Black, 128 GB)",128 GB ROM,12MP + 12MP | 12MP Front Camera,15.49 cm (6.1 inch) Liquid Retina HD Display,A13 Bionic Chip Processor,"₹41,990"
4,"SAMSUNG Galaxy F13 (Sunrise Copper, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,50MP + 5MP + 2MP | 8MP Front Camera,16.76 cm (6.6 inch) Full HD+ Display,6000 mAh Lithium Ion Battery,"₹9,499"
5,"realme 9i 5G (Metallica Gold, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,50MP + 2MP + 2MP | 8MP Front Camera,16.76 cm (6.6 inch) Full HD+ Display,5000 mAh Lithium Ion Battery,"₹12,999"
6,"POCO C31 (Royal Blue, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,13MP + 2MP + 2MP | 5MP Front Camera,16.59 cm (6.53 inch) HD+ Display,5000 mAh Lithium-ion Polymer Battery,"₹7,499"
7,"realme C33 (Sandy Gold, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,50MP + 0.3MP | 5MP Front Camera,16.51 cm (6.5 inch) HD+ Display,5000 mAh Lithium Ion Battery,"₹9,999"
8,"REDMI 10 (Pacific Blue, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,50MP + 2MP | 5MP Front Camera,17.02 cm (6.7 inch) HD+ Display,6000 mAh Lithium Polymer Battery,"₹8,999"
9,"realme C33 (Aqua Blue, 32 GB)",3 GB RAM | 32 GB ROM | Expandable Upto 1 TB,50MP + 0.3MP | 5MP Front Camera,16.51 cm (6.5 inch) HD+ Display,5000 mAh Lithium Ion Battery,"₹8,999"


# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [4]:
# first connect to the driver

driver = webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver_win32\chromedriver.exe")


In [5]:
# opening google maps
driver.get("https://www.google.co.in/maps")
time.sleep(3)

city = input('Enter City Name : ')                                         # Enter city to be searched
search = driver.find_element(By.id,"searchboxinput")                       # locating search bar
search.clear()                                                             # clearing search bar
time.sleep(2)
search.send_keys(city)                                                     # entering values in search bar
button = driver.find_element(By.id,"searchbox-searchbutton")               # locating search button
button.click()                                                             # clicking search button
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

Enter City Name : bhopal


AttributeError: type object 'By' has no attribute 'id'

# 6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21)from trak.in.


In [ ]:
driver = webdriver.Chrome(r"C:\Users\HP\Downloads\chromedriver_win32\chromedriver.exe")

In [ ]:
url1="https://trak.in/india-startup-funding-investment-2015/"

In [ ]:
driver.get(url1)

In [23]:
Dates=[]
Company=[]
Industry=[]
Investor_Name=[]
Investment_Type=[]
Amount=[]

In [ ]:
#scraping the company_name 
companies=driver.find_elements_by_xpath("//td[@class='column-3']")
for i in companies:
    if i.text is None :
        Company.append("--") 
    else:
        Company.append(i.text)
print(len(Company),Company)

In [ ]:
#scraping the Industry 
Ind=driver.find_elements_by_xpath("//td[@class='column-4']")
for i in Ind:
    if i.text is None :
        Industry.append("--") 
    else:
        Industry.append(i.text)
print(len(Industry),Industry)

In [24]:
#scraping the Dates 
dt=driver.find_elements_by_xpath("//td[@class='column-2']")
for i in dt:
    if i.text is None :
        Dates.append("--") 
    else:
        Dates.append(i.text)
print(len(Dates),Dates)

AttributeError: 'WebDriver' object has no attribute 'find_elements_by_xpath'

In [ ]:
#scraping the Investor_Name 
IN=driver.find_elements_by_xpath("//td[@class='column-7']")
for i in IN:
    if i.text is None :
        Investor_Name.append("--") 
    else:
        Investor_Name.append(i.text)
print(len(Investor_Name),Investor_Name)

In [ ]:
#scraping the Investment_Type 
IT=driver.find_elements_by_xpath("//td[@class='column-8']")
for i in IT:
    if i.text is None :
        Investment_Type.append("--") 
    else:
        Investment_Type.append(i.text)
print(len(Investment_Type),Investment_Type)

In [ ]:
#scraping the Amount 
Price=driver.find_elements_by_xpath("//td[@class='column-9']")
for i in Price:
    if i.text is None :
        Amount.append("--") 
    else:
        Amount.append(i.text)
print(len(Amount),Amount)

In [ ]:
Funding=pd.DataFrame([])
Funding['Company']=Company
Funding['Industry']=Industry
Funding['Investor_Name']=Investor_Name
Funding['Amount Invested']=Amount
Funding['Specification']=Investment_Type
Funding['Dates']=Dates
Funding

# 7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [ ]:
driver = webdriver.Chrome(r"E:\Aniket\chromedriver_win32\chromedriver.exe")

In [ ]:
url="https://www.digit.in/top-products/best-gaming-laptops-40.html"

In [ ]:
driver.get(url)

In [ ]:
Brands=[]
Products_Description=[]
Specification=[]
Price=[]

In [ ]:
br=driver.find_elements_by_xpath("//div[@class='TopNumbeHeading active sticky-footer']")
len(br)

In [ ]:
for i in br:
   
    Brands.append(str(i.text).replace("\n",""))
Brands

In [ ]:
sp=driver.find_elements_by_xpath("//div[@class='Specs-Wrap']")
len(sp)

In [ ]:
for i in sp:
   
    Specification.append(str(i.text).replace("\n",""))
Specification

In [ ]:
des=driver.find_elements_by_xpath("//div[@class='Section-center']")
len(des)

In [ ]:
for i in des:
   
    Products_Description.append(str(i.text).replace("\n",""))
Products_Description

In [ ]:
pri=driver.find_elements_by_xpath("//td[@class='smprice']")
len(pri)

In [ ]:
for i in pri:
   
    Price.append(str(i.text).replace("\n",""))
Price

In [ ]:
digit_lap=pd.DataFrame([])
digit_lap['Brands']=Brands[0:10]
digit_lap['Price']=Price[0:10]
digit_lap['Specification']=Specification[0:10]
digit_lap['Description']=Products_Description[0:10]
digit_lap